In [184]:
#!pip install selenium

In [395]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import json
import urllib.request
import os

MAIN_URL = "http://rsl.nstu.ru/"
login_name = ''
login_pwd = ''

video_path = ''
annotations_path = ''

In [396]:
# Вспомогательные функции
def elan_data_to_json(elan_data):
    elan_data_words = elan_data.split(';')[2]
    elan_data_words = elan_data_words[elan_data_words.index('['):]
    elan_data_words = ''.join(elan_data_words.split("\n"))
    elan_data_words_list = list()
    for x in elan_data_words.split('],['):
        word = x.replace("[","").replace("]","").replace('"','').split(',')
        if len(word)>2:
            word = [','.join(word[0:-2]), int(word[-2]), int(word[-1])]
        elan_data_words_list.append(word)
    return json.dumps(elan_data_words_list, ensure_ascii=False).encode("utf-8").decode()

In [397]:
# Вход на страницу с данными
options = Options()
driver = webdriver.Edge(options=options)
driver.get(MAIN_URL+'/login')
assert "Корпус русского жестового языка" in driver.title
elem = driver.find_element(By.NAME, 'LoginForm[username]')
elem.clear()
elem.send_keys(login_name)
elem.send_keys(Keys.RETURN)
elem = driver.find_element(By.NAME, 'LoginForm[password]')
elem.clear()
elem.send_keys(login_pwd)
elem.send_keys(Keys.RETURN)
driver.find_elements(By.XPATH, '//button')

[<selenium.webdriver.remote.webelement.WebElement (session="95a2a27b9ef99343e6fde2a962ce1cb1", element="E6BA697AE92F1E80BAE5A96AEF212A09_element_9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="95a2a27b9ef99343e6fde2a962ce1cb1", element="E6BA697AE92F1E80BAE5A96AEF212A09_element_10")>]

In [398]:
cities = ["Новосибирск", "Москва", "Узбекистан"]
months = ["Январь", "Февраль", "Март", "Апрель", "Май", "Июнь", "Июль", "Август", "Сентябрь", "Октябрь", "Ноябрь", "Декабрь"]

description = 'item_name;item_type;item_city;item_theme;item_year;item_month;respondent_id\n'

for i in range(1,13,1):
    CURRENT_URL = MAIN_URL + f'/data/index/ajax/yw0/Metadata_page/{i}'
    driver.get(CURRENT_URL)
    current_table = driver.find_element(By.TAG_NAME, "tbody")
    table_data = current_table.text
    table_data = table_data.split("\n")
    video_links = driver.find_elements(By.XPATH, "//a[@title='Просмотреть']")
    video_links = [x.get_property('href') for x in video_links if 'id' in x.get_property('href')]


    for item in table_data:
        item = item.split(' ')
        item_name = item[0]
        respondent_id = item[-1]
        if item[-2] in months:
            item_month = item[-2]
            item_year = item[-3]
            year_index = -3
        else:
            item_month = ''
            item_year = item[-2]
            year_index = -2
        
        for city in cities:
            if city in item:
                item_city = city
                city_index = item.index(city)
        item_type = ' '.join(item[1:city_index])
        item_theme = ' '.join(item[city_index+1:year_index])
        description += item_name+';'+item_type+';'+item_city+';'+item_theme+';'+item_year+';'+item_month+';'+respondent_id+'\n'
    
    # load_videos and glosses
    for video_link in video_links:
        driver.get(video_link)
        video_elem = driver.find_element(By.XPATH, "//video")
        load_video_link = video_elem.get_property('src')
        video_name = load_video_link.split('/')[-1]
        n=0
        while n<3:
            try:
                urllib.request.urlretrieve(load_video_link, video_path+video_name)
                n=3
            except:
                n+=1
        elan_name = video_name.split('.')[0]

        elan_elems = driver.find_elements(By.XPATH, "//script[@type='text/javascript']")
        elan_link = [x.get_property('src') for x in elan_elems if 'video' in x.get_property('src')]
        with urllib.request.urlopen(elan_link[0]) as f:
            myfile = f.read().decode('unicode-escape')
        with open(annotations_path+elan_name+'.txt', 'w') as new_file:
            new_file.write(elan_data_to_json(myfile))


C:\Users\pdkud\AppData\Local\Temp\ipykernel_23296\5122669.py:55: DeprecationWarning: invalid escape sequence '\/'
  myfile = f.read().decode('unicode-escape')


In [399]:
with open(annotations_path+'description.csv', 'w') as desc_file:
    desc_file.write(description)


In [400]:
driver.close()

In [404]:
len(os.listdir(video_path))

173

In [405]:
len(os.listdir(annotations_path))

174